# 5. Inference pipeline
This notebook consists of 3 parts:

1. Connecting to the Feature Store and retriving feature views/groups and model
3. Predicting if theres a detection or not
4. Saving the prediction in a new feature view/group

In [1]:
# import libraries
import pandas as pd
import hopsworks
import joblib

## 1. Connecting to the Feature Store and retriving feature views/groups and model

In [2]:
project = hopsworks.login(project="annikaij")
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# connect to the feature store
project = hopsworks.login(project="annikaij")
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Get the magnetic bike lane model from the model registry
mr = project.get_model_registry()
mag_bikelane_model = mr.get_model("bikelane_mag_hist_model", version=2)
model_dir = mag_bikelane_model.download()
mag_bikelane_model = joblib.load("/workspaces/2nd_semester_project/notebooks/models/bikelane_mag_hist_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Get the magnetic building model from the model registry
mag_building_model = mr.get_model("building_mag_hist_model", version=2)
model_dir = mag_building_model.download()
mag_building_model = joblib.load("/workspaces/2nd_semester_project/notebooks/models/building_mag_hist_model.pkl")

In [6]:
# get the radar bikelane model from the model registry
radar_bikelane_model = mr.get_model("bikelane_rad_hist_model", version=2)
model_dir = radar_bikelane_model.download()
radar_bikelane_model = joblib.load("/workspaces/2nd_semester_project/notebooks/models/bikelane_rad_hist_model.pkl")

In [7]:
# get the radar building model from the model registry
radar_building_model = mr.get_model("building_rad_hist_model", version=2)
model_dir = radar_building_model.download()
radar_building_model = joblib.load("/workspaces/2nd_semester_project/notebooks/models/building_rad_hist_model.pkl")

In [8]:
# Retrieve the feature views for each parking spot
mag_bikelane_feature_view = fs.get_feature_view(name="hist_bikelane_mag_fv", version=1)
mag_building_feature_view = fs.get_feature_view(name="hist_building_mag_fv", version=1)
rad_bikelane_feature_view = fs.get_feature_view(name="hist_bikelane_radar_fv", version=1)
rad_building_feature_view = fs.get_feature_view(name="hist_building_radar_fv", version=1)


In [9]:
# Make predictions on the newest magnetic bikelane data
mag_bikelane_data = mag_bikelane_feature_view.get_batch_data()
mag_bikelane_pred = mag_bikelane_model.predict(mag_bikelane_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.87s) 


In [10]:
# Make predictions on the newest magnetic building data
mag_building_data = mag_building_feature_view.get_batch_data()
mag_building_pred = mag_building_model.predict(mag_building_data)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.74s) 


In [11]:
# Make predictions on the newest radar bikelane data
rad_bikelane_data = rad_bikelane_feature_view.get_batch_data()
rad_bikelane_pred = radar_bikelane_model.predict(rad_bikelane_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.24s) 


In [12]:
# Make predictions on the newest radar building data
rad_building_data = rad_building_feature_view.get_batch_data()
rad_building_pred = radar_building_model.predict(rad_building_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.11s) 


In [14]:
# Add the predictions to the dataframes
mag_bikelane_data['mag_cluster'] = mag_bikelane_pred[mag_bikelane_pred.size-1]
mag_building_data['mag_cluster'] = mag_building_pred[mag_building_pred.size-1]
rad_bikelane_data['rad_cluster'] = rad_bikelane_pred[rad_bikelane_pred.size-1]
rad_building_data['rad_cluster'] = rad_building_pred[rad_building_pred.size-1]


In [17]:
#making a column describing if its data from bikelane or building
mag_bikelane_data['data_type'] = 'bikelane'
mag_building_data['data_type'] = 'building'
rad_bikelane_data['data_type'] = 'bikelane'
rad_building_data['data_type'] = 'building'

In [19]:
#combining the two dataframes
mag_data = pd.concat([mag_bikelane_data, mag_building_data])
rad_data = pd.concat([rad_bikelane_data, rad_building_data])

In [20]:
# upload the predictions to the feature store as a new feature group
latest_pred_fg = fs.get_or_create_feature_group(name="mag_parking_predictions",
                                  version=1,
                                  primary_key=["x", "y", "z"],
                                  description="Predictions for parking spots with magnetic data",
                                  online_enabled=False,
                                 )
latest_pred_fg.insert(mag_data)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549019/fs/544841/fg/849283


Uploading Dataframe: 0.00% |          | Rows 0/15260 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: mag_parking_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549019/jobs/named/mag_parking_predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7d8e75d67580>, None)

In [22]:
# upload the radar predictions to the feature store as a new feature group
latest_pred_fg = fs.get_or_create_feature_group(name="rad_parking_predictions",
                                  version=1,
                                  primary_key=['radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7'],
                                  description="Predictions for parking spots with radar data",
                                  online_enabled=False,
                                 )
latest_pred_fg.insert(rad_data)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549019/fs/544841/fg/850277


Uploading Dataframe: 0.00% |          | Rows 0/5313 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: rad_parking_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549019/jobs/named/rad_parking_predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7d8e75dcfd30>, None)